TODO:
- tenir compte des dernières frames

In [13]:
from ultralytics import YOLO
import cv2
import math
from tqdm import tqdm
import torch
import os
import moviepy.editor as mpe
import gc
import numpy as np
import sys
import uuid
import matplotlib.pyplot as plt

In [14]:
os.getcwd()

'/home/franzele/Desktop/univ_lille/m1s2/pji/pgm/test'

In [15]:
# def retrieve_audio(video):
#     video_clip = mpe.VideoFileClip(video)

#     # Extract the audio from the video clip
#     audio_clip = video_clip.audio
#     video_clip.close()

#     return audio_clip
# def combine_audio(video, audio, outname, fps):
#     final_clip = video.set_audio(audio)
#     # os.remove(outname)
#     print(type(final_clip))
#     final_clip.write_videofile(f"f{outname}", fps=fps)

In [16]:
def get_models(model_dir="models"):
    model_list = []
    for model_name in os.listdir(model_dir):
        model_list.append(YOLO(os.path.join(model_dir, model_name)))
    return model_list

In [17]:
# def variable_blur(data, kernel_size):
#   """ Blur with a variable window size
#   Parameters:
#   - data: 2D ndarray of floats or integers
#   - kernel_size: 2D ndarray of integers, same shape as data
#   Returns:
#   2D ndarray
#   """
#   data_blurred = np.empty(data.shape)
#   print(data_blurred)
#   Ni, Nj, Nk = data.shape
#   for i in range(Ni):
#     for j in range(Nj):
#       for k in range(Nk):

#         res = 0.0
#         weight = 0
#         sigma =  kernel_size[i, j, k]
#         for ii in range(i - sigma, i+sigma+1):
#           for jj in range(j - sigma, j+sigma+1):
#             if ii<0 or ii>=Ni or jj < 0 or jj >= Nj:
#               continue
#             res += data[ii, jj]
#             weight += 1
#         print(i, j, k)
#         data_blurred[i, j, k] = res/weight
#   return data_blurred

In [18]:
def blur_box(frame, box, min_conf=0.3):
    if math.ceil((box.conf[0]*100))/100 > min_conf:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        h, w = y2-y1, x2-x1

        ROI = frame[y1:y1+h, x1:x1+w]
        blur = cv2.GaussianBlur(ROI, (51,51), 0) 
        frame[y1:y1+h, x1:x1+w] = blur
        # frame[y1:y1+h, x1:x1+w] = np.zeros((h, w, 3))
    return frame

def read_video_file(video_path):
    vidcap = cv2.VideoCapture(video_path)
    return vidcap, \
            int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)), \
            vidcap.get(cv2.CAP_PROP_FPS), \
            (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

def create_video_file(output_path, fps, frame_size):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(filename=output_path, 
                        fourcc=fourcc, 
                        fps=fps, 
                        frameSize=frame_size)
    return video

In [19]:
def tmp_video(orig_video, output_video, frame_rate):
    vidcap, frame_number, fps, frame_size = read_video_file(orig_video)
    video = create_video_file(output_video, fps, frame_size)
    success, img = vidcap.read()
    i=0
    while success:
        if i%frame_rate == 0:
            video.write(img)
        i+=1
        success, img = vidcap.read()
    # on prends la dernière frame
    if (i-1)%frame_rate != 0:
        video.write(img)
    del i
    del img
    del success
    vidcap.release
    video.release()
    gc.collect()

In [20]:
def transform1(model, video_path, output_path, frame_verif_rate=5, verbose=False):
    if verbose: print("Création vidéo temporaire")
    tmp_video_name = f"{uuid.uuid4().hex}.mp4"
    tmp_video(video_path, tmp_video_name, frame_verif_rate)
    gc.collect()

    if verbose: print("Floutage")
    vidcap_s, frame_number_s, _, _ = read_video_file(tmp_video_name)
    vidcap_o, _, fps_o, frame_size_o = read_video_file(video_path)
    video = create_video_file(output_path, fps_o, frame_size_o)
    success_s, img_s = vidcap_s.read()
    assert success_s, "Error while loading the temp video"

    for i in tqdm(range(frame_number_s)):
        results_s = model(img_s, verbose=False, stream=True)
        to_blur = len(next(results_s).boxes.conf) > 0
        del results_s ; gc.collect()
        # première frame
        if i == 0:
            for j in range(math.ceil(frame_verif_rate/2)):
                success_o, img_o = vidcap_o.read()
                if to_blur:
                    boxes = next(model(img_o, verbose=False, stream=True)).boxes
                    for box in boxes:
                        img_o = blur_box(img_o, box)
                video.write(img_o)
        # autre frame
        else:
            for j in range(frame_verif_rate):
                success_o, img_o = vidcap_o.read()
                if to_blur:
                    boxes = next(model(img_o, verbose=False, stream=True)).boxes
                    for box in boxes:
                        img_o = blur_box(img_o, box)
                video.write(img_o)
            # dernières frames
            if i+1 == frame_number_s:
                success_o, img_o = vidcap_o.read()
                while success_o:
                    video.write(img_o)
                    if to_blur:
                        boxes = next(model(img_o, verbose=False, stream=True)).boxes
                        for box in boxes:
                            img_o = blur_box(img_o, box)
                    success_o, img_o = vidcap_o.read()

        success_s, img_s= vidcap_s.read()

    vidcap_s.release()
    vidcap_o.release()
    video.release()
    if verbose: print("Suppression vidéo temporaire")
    os.remove(tmp_video_name)

    if verbose: print("========= FIN =========")

In [21]:
def transform2(model_list, video_path, output_path):
    vidcap, frame_number, fps, frame_size = read_video_file(video_path)
    video = create_video_file(output_path, fps, frame_size)

    results = [model(video_path, verbose=True) for model in model_list]
    mean_conf = torch.tensor([[i.conf for x in r for i in x.boxes] for r in results]).mean(1).item()
    conf_tol = mean_conf * 0.75
    interesting_frame = torch.tensor([[(x.boxes.conf > conf_tol).any().item() for x in r] for r in results]).any(0).tolist()
    
    success, img = vidcap.read()
    if success:
        for i in tqdm(range(frame_number)):
            if interesting_frame[i]:
                for result_id in range(len(model_list)):
                    frame_result = results[result_id][i]
                    for r in frame_result:
                        boxes = r.boxes
                        for box in boxes:
                            img = blur_box(img, box, conf_tol)

            video.write(img)
            success, img = vidcap.read()
    vidcap.release()
    video.release()

In [22]:
# model_list = get_models()
model_list = [
    YOLO("models/yolov8n-face.pt"),
]

In [23]:
transform1(model_list[0], "pietons2.mp4", "video.mp4", frame_verif_rate=12, verbose=True)

Création vidéo temporaire
Floutage


100%|██████████| 45/45 [00:19<00:00,  2.28it/s]

Suppression vidéo temporaire
========= FIN =========


In [24]:
# transform2(model_list, "action_bronson.mp4", "video2.mp4")